In [343]:
import pandas as pd
import numpy as np
import glob
import ast
import json
import scipy

import re
import string
import nltk
from nltk.stem import WordNetLemmatizer 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import train_test_split
#from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, r2_score, roc_auc_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score

import lightfm
from lightfm import LightFM

#import Reviews_combiner_processer
#from Reviews_combiner_processer import user_info
import ast


In [2]:
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate

from surprise import NormalPredictor

from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline

from surprise import SVD
from surprise import SVDpp

from surprise import NMF
from surprise import SlopeOne, CoClustering, BaselineOnly
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import GridSearchCV

from collections import defaultdict

from lightfm import cross_validation
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import recall_at_k
from lightfm.evaluation import auc_score

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

## Ratings

In [4]:
df_rating = pd.read_csv('product_ratings_customer.csv')

## Product

In [256]:
df_pro.head()

,Unnamed: 0,Product_pid,brand,product_name,product_type,stars,Awards,Clinical_tested,Skintype_normal,Skintype_dry,...,Skinc_Wrinkles,Skinc_Dullness,Skinc_Pores,Skinc_Uneven_Texture,Skinc_Acne,Skinc_Uneven_Tone,Skinc_Dark_Spot,ingredients,ingred_list,Skinc_total
0,0,P57002,fresh,Sugar Lip Balm Sunscreen SPF 15,Lip Balms & Treatments,89.474,1,0,1,1,...,0,0,0,0,0,0,0,"Cera Alba (Beeswax), Ricinus Communis (Castor)...","['Cera Alba (Beeswax)', 'Ricinus Communis (Cas...",0
1,1,P462699,Glow Recipe,Plum Plump™ Hyaluronic Acid Serum,Face Serums,91.804,0,1,1,1,...,1,0,0,0,0,0,0,"Water, Glycerin, Propanediol, Pentylene Glycol...","['Water', 'Glycerin', 'Propanediol', 'Pentylen...",2
2,2,P463948,SEPHORA COLLECTION,Holographic Effect Mask,Face Masks,82.858,0,0,1,1,...,0,0,0,0,0,0,0,"Aqua (Water), Glycerin, Butylene Glycol, Cerat...","['Aqua (Water)', 'Glycerin', 'Butylene Glycol'...",0
3,3,P439926,Drunk Elephant,A-Passioni™ Retinol Cream,Face Serums,82.162,1,1,1,1,...,1,0,0,0,0,0,1,"Water, Glycerin, Coco-Caprylate, Stearic Acid,...","['Water', 'Glycerin', 'Coco-Caprylate', 'Stear...",2
4,4,P447791,Glow Recipe,Avocado Melt Retinol Eye Sleeping Mask,Eye Creams & Treatments,84.896,0,1,1,1,...,0,0,0,0,0,0,0,"Water, Glycerin, Propanediol, Glyceryl Stearat...","['Water', 'Glycerin', 'Propanediol', 'Glyceryl...",1


In [5]:
df_pro = pd.read_csv('product_post_eda.csv')

In [6]:
# df_product_select_clean.join(df_product_select['details']).to_csv('Final_product_df.csv')

#  NLP for reviews

In [7]:
df_nlp = df_rating.drop_duplicates(subset=['UserNickname', 'ProductId']).dropna(subset=['UserNickname']).reset_index(drop=True)

In [8]:
len(df_nlp)

312562

In [9]:
stopwords = nltk.corpus.stopwords.words('english')
lemma = WordNetLemmatizer()

In [10]:
def review_text_lemma(reviewText):
    reviews = "".join([w for w in reviewText if w not in string.punctuation])
    tokens = re.split('\W+', reviews)
    review_clean = [lemma.lemmatize(ws) for ws in tokens if ws not in stopwords]
    review_cleaner = " ".join(review_clean)
    return review_cleaner

In [11]:
df_nlp['review_kw'] = df_nlp['ReviewText'].apply(lambda x: review_text_lemma(str(x).lower()))
df_nlp['title_kw'] = df_nlp['Title'].apply(lambda x: review_text_lemma(str(x).lower()))

In [12]:
df_rating_clean = df_nlp.drop(['Badges', 'BadgesOrder', 'ContentLocale',
            'ContextDataValues', 'ContextDataValuesOrder', 'eyecolor', 'haircolor'], axis=1)

In [13]:
df_rating_clean.skintype.value_counts()

Combination    139327
No info         58963
Dry             49200
Normal          33874
Oily            31198
Name: skintype, dtype: int64

In [14]:
df_rating_userskintype = df_nlp[['UserNickname', 'skintype']]

In [15]:
df_rating_userskintype.head()

,UserNickname,skintype
0,lizziebae,Combination
1,BelleOh,Combination
2,BelLaCafe,Dry
3,bluekrait,Combination
4,fairginger,No info


In [16]:
df_nlp.columns

Index(['Unnamed: 0', 'Badges', 'BadgesOrder', 'ContentLocale',
       'ContextDataValues', 'ContextDataValuesOrder', 'Id', 'IsRatingsOnly',
       'IsRecommended', 'ProductId', 'Rating', 'ReviewText', 'SubmissionId',
       'SubmissionTime', 'Title', 'TotalNegativeFeedbackCount',
       'TotalPositiveFeedbackCount', 'UserNickname', 'Dict_context',
       'skintype', 'eyecolor', 'skinconcerns', 'haircolor', 'skintone',
       'review_kw', 'title_kw'],
      dtype='object')

In [17]:
tfidf = TfidfVectorizer(max_features = 8000)
X = tfidf.fit_transform(df_nlp['review_kw']).toarray()[:5000]
y = df_nlp['Rating'][:5000]

In [18]:
X.shape, y.shape

((5000, 8000), (5000,))

In [19]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=12345)

In [20]:
X_train.shape, y_train.shape

((3500, 8000), (3500,))

In [21]:
model = RandomForestClassifier(n_estimators = 20, 
                            criterion = 'entropy') 
                              
model.fit(X_train, y_train) 

RandomForestClassifier(criterion='entropy', n_estimators=20)

In [22]:
y_pred = model.predict(X_test) 
from sklearn.metrics import confusion_matrix 
  
cm = confusion_matrix(y_test, y_pred) 
cm 

array([[ 12,   2,   0,   1,  78],
       [  3,   5,   2,   4,  72],
       [  3,   2,   1,  12,  90],
       [  0,   0,   0,  15, 225],
       [  0,   0,   2,   8, 963]])

# Surprise Remmender System

In [349]:
from surprise.model_selection import train_test_split

In [350]:
df_vanilla = pd.read_csv('vanilla_data.csv')

In [351]:
df_vanilla.head()

,Unnamed: 0,UserNickname,ProductId,Rating
0,0,lizziebae,P420224,5
1,1,BelleOh,P420224,5
2,2,BelLaCafe,P420224,5
3,3,bluekrait,P420224,5
4,4,fairginger,P420224,5


In [352]:
df_vanilla_clean = df_vanilla.drop_duplicates(subset=['UserNickname', 'ProductId']).dropna(subset=['UserNickname']).reset_index(drop=True)

In [353]:
len(df_vanilla), len(df_vanilla_clean)

(316548, 312562)

In [354]:
len(df_vanilla_clean)

312562

In [355]:
df_binary = df_vanilla_clean
df_binary['Rating_bin'] = np.where(df_binary['Rating']>2, 1, 0).astype(int)

In [356]:
df_vanilla_matrix = df_vanilla_clean.pivot(index='UserNickname', 
                    columns='ProductId', values = 'Rating').fillna(0)

In [357]:
df_vanilla_matrix_binary = df_binary.pivot(index='UserNickname', 
                    columns='ProductId', values = 'Rating_bin').fillna(0)

In [358]:
# Regular rating stars dataset
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(df_vanilla_clean[['UserNickname', 'ProductId', 'Rating']], reader)
data_bin = Dataset.load_from_df(df_binary[['UserNickname', 'ProductId', 'Rating_bin']], reader)

In [34]:
# SVD model
SVD_res = []
# Iterate over all algorithms
for algo in [SVD(), SVDpp()]:
    
    # Perform cross validation
    cv_fold = cross_validate(algo, data_bin, measures=['RMSE'], cv=5, verbose=False)
    
    # Get results & append algorithm name
    results = pd.DataFrame.from_dict(cv_fold).mean(axis=0)
    results = results.append(pd.Series([str(algo).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    SVD_res.append(results)

surprise_results_svd = pd.DataFrame(SVD_res).set_index('Algorithm').sort_values('test_rmse')
surprise_results_svd

,test_rmse,fit_time,test_time
Algorithm,,,
SVDpp,0.320670,36.592079,0.815952
SVD,0.322079,18.412205,0.551274


In [35]:
# Normal Predictor & Slope One

NP_Slope_res = []
# Iterate over all algorithms
for algo in [NormalPredictor(), SlopeOne()]:
    
    # Perform cross validation
    cv_fold = cross_validate(algo, data_bin, measures=['RMSE'], cv=5, verbose=False)
    
    # Get results & append algorithm name
    results = pd.DataFrame.from_dict(cv_fold).mean(axis=0)
    results = results.append(pd.Series([str(algo).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    NP_Slope_res.append(results)

surprise_results_nmp_slope = pd.DataFrame(NP_Slope_res).set_index('Algorithm').sort_values('test_rmse')
surprise_results_nmp_slope

,test_rmse,fit_time,test_time
Algorithm,,,
SlopeOne,0.355642,2.573128,0.691683
NormalPredictor,0.410729,0.535604,0.578424


In [36]:
# Co-Clustering
CoC_res = []
# Iterate over all algorithms
for algorithm in [CoClustering(), BaselineOnly()]:
    
    # Perform cross validation
    cv_fold = cross_validate(algorithm, data_bin, measures=['RMSE'], cv=5, verbose=False)
    
    # Get results & append algorithm name
    results = pd.DataFrame.from_dict(cv_fold).mean(axis=0)
    results = results.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    CoC_res.append(results)

surprise_results_coc = pd.DataFrame(CoC_res).set_index('Algorithm').sort_values('test_rmse')
surprise_results_coc

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


,test_rmse,fit_time,test_time
Algorithm,,,
BaselineOnly,0.320764,1.586316,0.634806
CoClustering,0.351816,28.563486,0.523282


### Baseline Only

In [37]:
# Baseline only and SVDpp are the best
# Grid Search for Baseline only first

algo = BaselineOnly()

param_grid = {'bsl_options':{ 'method': ['als', 'sgd']} } 
      #      'n_epochs': [5, 10, 15],
       #     'reg_u': [5, 10, 15],
       #     'reg_i': [5, 10, 15]}}
              
gs = GridSearchCV(BaselineOnly, param_grid, measures=['rmse', 'mae'], cv=5)

gs.fit(data_bin)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
0.3206515253594061
{'bsl_options': {'method': 'sgd'}}


In [38]:
algo = BaselineOnly()

param_grid = {'bsl_options':{ 'method': ['sgd'],
                             
      'n_epochs': [15, 20, 25],
       'reg': [0.02, 0.2, 2],
       'reg_i': [0.005, 0.5]}}
              
gs = GridSearchCV(BaselineOnly, param_grid, measures=['rmse', 'mae'], cv=5)

gs.fit(data_bin)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimati

In [33]:
# Using Baseline Only to make predictions
trainset, testset = train_test_split(data_bin, test_size = 0.3)
alg = BaselineOnly(bsl_options = { 'method':'sgd', 'n_epochs': 25, 'reg':0.2, 'reg_i':0.005})
alg.fit(trainset)
test_pred = alg.test(testset)

Estimating biases using sgd...


In [34]:
real = []
pred = []
for i in range(len(testset)):
    r = testset[i][2]
    p = test_pred[i][3]
    real.append(r)
    pred.append(p)

In [35]:
# Predictions are continuous
# Used 0.5 as a threshold, >0.5 as recommending, <0.5 as not recommending
pred_bin = []
for i in range(len(test_pred)):
    predbin = np.where(test_pred[i][3]>0.6, 1, 0)
    pred_bin.append(predbin)

In [36]:
auc_bo = accuracy_score(real, pred_bin)
prec_bo = precision_score(real, pred_bin)
rec_bo = recall_score(real, pred_bin)
print('Accuracy Score:', '{:.3f}'.format(auc_bo), 
      '\nPrecision Score:', '{:.3f}'.format(prec_bo), 
      '\nRecall Score:', '{:.3f}'.format(rec_bo))

Accuracy Score: 0.877 
Precision Score: 0.879 
Recall Score: 0.997


### SVD pp

In [43]:
param_grid = {'n_epochs': [15, 20, 25], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=5)

gs.fit(data_bin)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

0.32065294829407914
{'n_epochs': 25, 'lr_all': 0.005, 'reg_all': 0.4}


In [359]:
trainset, testset = train_test_split(data_bin, test_size = 0.3)
alg = SVDpp(n_epochs = 25, lr_all = 0.005, reg_all = 0.4)
alg.fit(trainset)
test_pred = alg.test(testset)

In [360]:
real = []
pred = []
for i in range(len(testset)):
    r = testset[i][2]
    p = test_pred[i][3]
    real.append(r)
    pred.append(p)

In [361]:
# Predictions are continuous
# Used 0.5 as a threshold, >0.5 as recommending, <0.5 as not recommending
pred_bin = []
for i in range(len(test_pred)):
    predbin = np.where(test_pred[i][3]>0.6, 1, 0)
    pred_bin.append(predbin)

In [362]:
acu_bo = accuracy_score(real, pred_bin)
auc_bo = roc_auc_score(real, pred_bin, average='weighted')
prec_bo = precision_score(real, pred_bin)
rec_bo = recall_score(real, pred_bin)
print('Accuracy Score:', '{:.3f}'.format(acu_bo), 
      '\nROC AUC Score:', '{:.3f}'.format(auc_bo),
      '\nPrecision Score:', '{:.3f}'.format(prec_bo), 
      '\nRecall Score:', '{:.3f}'.format(rec_bo))

Accuracy Score: 0.876 
ROC AUC Score: 0.505 
Precision Score: 0.878 
Recall Score: 0.998


### SVDpp and Reducing my initial dataset
some people only rated a product once and that's difficult to predict. So I'm going to get rid of those users

In [363]:
df_vanilla_clean.UserNickname.value_counts()[:51813]

jen101            81
faeriegirl        45
itsryanwithani    44
Mitz17            43
JennRen44         42
                  ..
catsandblack       2
luvsephora77       2
kaylabeth90        2
dlynn009           2
maskofshelley      1
Name: UserNickname, Length: 51813, dtype: int64

In [364]:
users_sample = [i for i in df_vanilla_clean.UserNickname.value_counts()[:51813].to_frame().index]

In [365]:
df_vanilla_sample = df_vanilla_clean[df_vanilla_clean.UserNickname.isin(users_sample)]


In [366]:
df_vanilla_sample['Rating_bin'] = np.where(df_vanilla_sample['Rating']>2, 1, 0).astype(int)

<ipython-input-366-4705d85d8202>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vanilla_sample['Rating_bin'] = np.where(df_vanilla_sample['Rating']>2, 1, 0).astype(int)


In [367]:
df_vanilla_sample_matrix = df_vanilla_sample.pivot(index='UserNickname', 
                    columns='ProductId', values = 'Rating_bin').fillna(0)

In [368]:
reader = Reader(rating_scale=(0, 1))
datasample_bin = Dataset.load_from_df(df_vanilla_sample[['UserNickname', 'ProductId', 'Rating_bin']], reader)

In [369]:
# Building SVDpp model

trainset, testset = train_test_split(datasample_bin, test_size = 0.3)
alg = SVDpp(n_epochs = 25, lr_all = 0.005, reg_all = 0.4)
alg.fit(trainset)
test_pred = alg.test(testset)

In [370]:
real = []
pred = []
for i in range(len(testset)):
    r = testset[i][2]
    p = test_pred[i][3]
    real.append(r)
    pred.append(p)

# Predictions are continuous
# Used 0.5 as a threshold, >0.5 as recommending, <0.5 as not recommending
pred_bin = []
for i in range(len(test_pred)):
    predbin = np.where(test_pred[i][3]>0.6, 1, 0)
    pred_bin.append(predbin)

In [371]:
acu_bo = accuracy_score(real, pred_bin)
auc_bo = roc_auc_score(real, pred_bin, average='weighted')
prec_bo = precision_score(real, pred_bin)
rec_bo = recall_score(real, pred_bin)
print('Accuracy Score:', '{:.3f}'.format(acu_bo), 
      '\nROC AUC Score:, ''{:.3f}'.format(auc_bo),
      '\nPrecision Score:', '{:.3f}'.format(prec_bo), 
      '\nRecall Score:', '{:.3f}'.format(rec_bo))

Accuracy Score: 0.882 
ROC AUC Score:, 0.507 
Precision Score: 0.883 
Recall Score: 0.998


In [220]:
# Getting a sample of 50 users
ta_1 = defaultdict(list)

for iid in df_vanilla_sample_matrix.columns:
    df_1 = pd.DataFrame()
    for uid in df_vanilla_sample_matrix.index[:50]:
        est = alg.predict(uid, iid)
        ta_1[uid].append(est)
        a = pd.DataFrame(ta_1[uid], columns = ['uid', 'iid', 'r_ui', 'est', 'details'])
        df_1 = pd.concat([a, df_1])
df_1.to_csv('prediction_1.csv')

In [225]:
df_vanilla_clean_13bananas = df_vanilla_clean[df_vanilla_clean.UserNickname == '13bananas']
df_vanilla_clean_13bananas

,Unnamed: 0,UserNickname,ProductId,Rating,Rating_bin
119043,120735,13bananas,P442842,5,1
298056,301877,13bananas,P442846,5,1


In [277]:
user_50 = pd.read_csv('prediction_1.csv')
user_50_compare = user_50.merge(df_vanilla_clean, left_on = ['uid', 'iid'],
                    right_on = ['UserNickname', 'ProductId'], how = 'left')
user_50_compare = user_50_compare.drop(['Unnamed: 0_x', 'r_ui', 'details', 
                        'Unnamed: 0_y', 'UserNickname', 'ProductId', 'Rating'], axis=1)
user_50_compare['Rating_bin'] = user_50_compare['Rating_bin'].fillna(-1)
user_50_compare.uid.unique()

array(['13bananas', '13Napier', '1337', '1331', '131oz', '1273natkb',
       '124598', '123lauryn', '123cantho', '123beautyinside', '123Mandy',
       '123Beautylove', '123AllyC', '1234kat', '1234Anna', '1234Abby',
       '1234567899', '11Makeup11', '118makeupjunkie', '1125dmw', '110011',
       '1083', '103368', '100natalia', '100Pris', '1000attempts', '1-jan',
       '0verthestars', '0tter99', '0smoky0', '0nicolee', '0l1ve',
       '0clare0', '0Brielle', '093008130306', '090913', '07Amanda',
       '0508angela1995', '0415KM', '0392amanda', '0305lovelyone',
       '00hatter', '00avancer', '00Nicole', '007gal', '007CatWomen',
       '007Barbie', '005leafs', '001000101', '00022'], dtype=object)

In [309]:
## Making a function for getting new recommendations
def new_recommendations(userid, database_pred):
    df_lookup = database_pred[database_pred.uid.isin([userid])].sort_values(by=['Rating_bin', 'est'], ascending=False)
    df_purchased = df_lookup[df_lookup.Rating_bin == 1]
    df_topnew = df_lookup[(df_lookup.Rating_bin == -1)][:5]
    
    purchased = []
    
    new_rec = []
    
    product_name_purchased = df_pro[df_pro.Product_pid.isin(df_purchased['iid'].unique())]
    
    product_name_purchased['complete'] = product_name_purchased['brand'] + " - " + product_name_purchased['product_name']
    
    purchased.append(product_name_purchased['complete'])
    
    product_name_new = df_pro[df_pro.Product_pid.isin(df_topnew['iid'].unique())]
    
    product_name_new['complete'] = product_name_new['brand'] + " - " + product_name_new['product_name']
    
    new_rec = [product_name_new['complete']]
    
    print('Hello!', userid)
    print('\nIt seems you like:\n', '\n', purchased[:][0])
    print('\nWe think you might also like \n', '\n', new_rec[:][0])
    

In [310]:
new_recommendations('118makeupjunkie', user_50_compare)

Hello! 118makeupjunkie

It seems you like:
 
 91     Herbivore - Blue Tansy BHA and Enzyme Pore Ref...
101                             Caudalie - Beauty Elixir
492       Herbivore - Citrine Glowing Hydration Body Oil
Name: complete, dtype: object

We think you might also like 
 
 264            fresh - Hesperides Grapefruit Body Lotion
475    Skinfix - Redness Recovery+ Antioxidant Eye Tr...
536            Saint Jane Beauty - Luxury CBD Body Serum
646         KORRES - Greek Yoghurt Calming + Cooling Gel
647                     Omorovicza - Moor Cream Cleanser
Name: complete, dtype: object


<ipython-input-309-ddc386cf1543>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_name_purchased['complete'] = product_name_purchased['brand'] + " - " + product_name_purchased['product_name']
<ipython-input-309-ddc386cf1543>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_name_new['complete'] = product_name_new['brand'] + " - " + product_name_new['product_name']


# LightFM

In [311]:
df_product_select = pd.read_csv('Final_product_df.csv')

In [312]:
df_product_fts = df_product_select[['Product_pid', 'Awards', 'Clinical_tested']]

In [313]:
df_product_matrix = df_product_fts.set_index('Product_pid').sort_index(axis=0)

In [314]:
df_product_matrix.head()

,Awards,Clinical_tested
Product_pid,,
P100101,0,0
P12045,0,1
P12295,0,1
P12336,0,1
P152203,0,0


In [315]:
data = scipy.sparse.csr_matrix(df_vanilla_matrix)
data_bin = scipy.sparse.csr_matrix(df_vanilla_matrix_binary)

## Vanilla 

In [316]:
# Vanilla
# Original Rating
train, test = cross_validation.random_train_test_split(data, test_percentage=0.3, random_state=12345)
model = LightFM(loss = 'warp', no_components = 8)
model.fit(train, epochs=25)
print(auc_score(model, test, train).mean())
print(recall_at_k(model, test, train).mean())
print(precision_at_k(model, test, train).mean())

0.7956745
0.1646646508093749
0.018723162


In [317]:
# Vanilla binary data
# Rating 1 or 2 will be 0
train, test = cross_validation.random_train_test_split(data_bin, test_percentage=0.3, random_state=12345)
model = LightFM(loss = 'warp', no_components = 8)
model.fit(train, epochs=25)
print(auc_score(model, test, train).mean())
print(recall_at_k(model, test, train).mean())
print(precision_at_k(model, test, train).mean())

0.8030535
0.17642505577734202
0.01975212


## Adding user features only

In [318]:
df_rating_users = df_rating[['UserNickname', 'ProductId', 'Rating', 'skintype', 'skinconcerns', 'skintone']]

In [319]:
df_rating_cleanuser = df_rating_users.drop_duplicates(subset=['UserNickname', 'ProductId']).dropna(subset=['UserNickname']).reset_index(drop=True)

In [320]:
data_user_item = df_rating_cleanuser.pivot(index='UserNickname', 
                    columns='ProductId', values = 'Rating').fillna(0)

In [321]:
user_feature = pd.get_dummies(df_rating_cleanuser[['skintype', 'skinconcerns', 'skintone']]).reset_index(drop=True)
df_user_feature = df_rating_cleanuser[['UserNickname']].join(user_feature).drop_duplicates(subset=['UserNickname'])
user_matrix = df_user_feature.set_index('UserNickname').sort_index(axis=0)

In [322]:
userfeature = scipy.sparse.csr_matrix(user_matrix)

In [323]:
# Model adding user feature
train, test = cross_validation.random_train_test_split(data_bin, test_percentage=0.3, random_state=12345)
model = LightFM(loss = 'warp', no_components = 8)
model.fit(train, user_features = userfeature, epochs=25)
print(auc_score(model, test, train_interactions=train, user_features = userfeature).mean())
print(recall_at_k(model, test, train_interactions=train, user_features = userfeature).mean())
print(precision_at_k(model, test, train_interactions=train, user_features = userfeature).mean())

0.8728628
0.21176798242409894
0.023229467


In [347]:
# Model adding user feature and excluding users with only one reviews
sample_bin = scipy.sparse.csr_matrix(df_vanilla_sample_matrix)
train, test = cross_validation.random_train_test_split(sample_bin, test_percentage=0.3, random_state=12345)
model = LightFM(loss = 'warp', no_components = 8)
model.fit(train, user_features = userfeature, epochs=25)
print(auc_score(model, test, train_interactions=train, user_features = userfeature).mean())
print(recall_at_k(model, test, train_interactions=train, user_features = userfeature).mean())
print(precision_at_k(model, test, train_interactions=train, user_features = userfeature).mean())

0.83250934
0.13134598027930178
0.017620146


## Adding item features only

In [107]:
# Processing products info
df_pro.columns
df_product_select_clean = df_pro

In [108]:
df_product_select_clean['all_kw'] = df_product_select_clean[['brand', 'product_name', 
                                                             'product_type', 'ingred_list']].agg(','.join, axis=1)

In [109]:
df_prod_m_c = df_product_select_clean[['Product_pid','all_kw']].set_index('Product_pid').sort_index(axis=0)

### TF-IDF Vector, LightFM

In [110]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')

In [111]:
matrix_prod = tf.fit_transform(df_prod_m_c['all_kw'])

In [112]:
itemfeature = scipy.sparse.csr_matrix(matrix_prod)

In [113]:
train, test = cross_validation.random_train_test_split(data_bin, test_percentage=0.3, random_state=12345)
model = LightFM(loss = 'warp', no_components = 8)
model.fit(train, item_features = itemfeature, epochs=25)
print(auc_score(model, test, train_interactions=train,
            item_features = itemfeature).mean())
print(recall_at_k(model, test, train_interactions=train, 
            item_features = itemfeature).mean())
print(precision_at_k(model, test, train_interactions=train, 
            item_features = itemfeature).mean())

0.78690624
0.17197407735850423
0.019662488


### Count Vector, LightFM

In [115]:
count = CountVectorizer()
count_matrix = count.fit_transform(df_product_select_clean['all_kw'])

# generating the cosine similarity matrix
# cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [116]:
model = LightFM(loss = 'warp', no_components = 8)
model.fit(train, item_features = count_matrix, epochs=25)
print(auc_score(model, test, train_interactions=train,
            item_features = count_matrix).mean())

print(recall_at_k(model, test, train_interactions=train, 
            item_features = count_matrix).mean())
print(precision_at_k(model, test, train_interactions=train, 
            item_features = count_matrix).mean())

0.8101676
0.8759591
0.13897170258104366
0.0152482325


## TF-IDF, pairwise

In [121]:
df_rating = pd.read_csv('product_ratings_customer.csv')
df_rating['skintype'] = df_rating['skintype'].replace('No info', 'Unknown')
df_rating['skinconcerns'] = df_rating['skinconcerns'].replace('No info', 'Unknown')
df_rating['skintone'] = df_rating['skintone'].replace('No info', 'Unknown')
df_rating['all_kw'] = df_rating[['skintype', 'skintone', 'skinconcerns']].agg(','.join, axis=1)

In [122]:
df_rating.head()

,Unnamed: 0,Badges,BadgesOrder,ContentLocale,ContextDataValues,ContextDataValuesOrder,Id,IsRatingsOnly,IsRecommended,ProductId,...,TotalPositiveFeedbackCount,UserNickname,Dict_context,skintype,eyecolor,skinconcerns,haircolor,skintone,review_kw,all_kw
0,0,{},[],en_US,"{'skinType': {'Value': 'combination', 'Id': 's...","['eyeColor', 'hairColor', 'skinTone', 'skinTyp...",117129210,False,1.0,P420224,...,10,lizziebae,"{'skinType': {'Value': 'combination', 'Id': 's...",Combination,Green,Acne,Blonde,Fair,"['really', 'liked', 'product', 'made', 'skin',...","Combination,Fair,Acne"
1,1,{},[],en_US,"{'skinType': {'Value': 'combination', 'Id': 's...","['eyeColor', 'hairColor', 'skinTone', 'skinTyp...",156143419,False,1.0,P420224,...,7,BelleOh,"{'skinType': {'Value': 'combination', 'Id': 's...",Combination,Brown,Aging,Blonde,Fair,"['using', 'essence', 'recently', 'ran', 'notic...","Combination,Fair,Aging"
2,2,{},[],en_US,"{'skinType': {'Value': 'dry', 'Id': 'skinType'...","['eyeColor', 'hairColor', 'skinTone', 'skinTyp...",137182034,False,1.0,P420224,...,1,BelLaCafe,"{'skinType': {'Value': 'dry', 'Id': 'skinType'...",Dry,Brown,Acne,Brunette,Olive,"['lightly', 'exfoliates', 'surface', 'skin', '...","Dry,Olive,Acne"
3,3,{},[],en_US,"{'skinType': {'Value': 'combination', 'Id': 's...","['eyeColor', 'hairColor', 'skinTone', 'skinTyp...",126600083,False,1.0,P420224,...,3,bluekrait,"{'skinType': {'Value': 'combination', 'Id': 's...",Combination,Brown,Acne,Brunette,Medium,"['vinoperfect', 'line', 'diminished', 'dark', ...","Combination,Medium,Acne"
4,4,{},[],en_US,{},[],104184861,False,1.0,P420224,...,4,fairginger,{},Unknown,No info,Unknown,No info,Unknown,"['literally', 'holy', 'grail', 'there', 'notic...","Unknown,Unknown,Unknown"


In [126]:
df_rating = df_rating.sort_values(by=['UserNickname', 'all_kw']).reset_index(drop=True).dropna(subset=['UserNickname'])
df_rating = df_rating.drop_duplicates(subset=['UserNickname'], keep='first').reset_index(drop=True)
df_rating_user = df_rating[['UserNickname', 'all_kw']]

In [129]:
# Item similarities
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
matrix_prod = tf.fit_transform(df_prod_m_c['all_kw'])
cosine_sim2 = cosine_similarity(matrix_prod, matrix_prod)

In [230]:
cosine_sim2

array([[1.00000000e+00, 1.15265316e-01, 8.45969825e-02, ...,
        9.00738814e-02, 3.43559169e-02, 2.01729159e-02],
       [1.15265316e-01, 1.00000000e+00, 4.60154416e-02, ...,
        2.07237874e-03, 4.72178723e-03, 9.21458555e-03],
       [8.45969825e-02, 4.60154416e-02, 1.00000000e+00, ...,
        1.03733604e-01, 4.62912716e-02, 2.20831903e-02],
       ...,
       [9.00738814e-02, 2.07237874e-03, 1.03733604e-01, ...,
        1.00000000e+00, 6.02581902e-02, 3.68133045e-04],
       [3.43559169e-02, 4.72178723e-03, 4.62912716e-02, ...,
        6.02581902e-02, 1.00000000e+00, 2.30351099e-02],
       [2.01729159e-02, 9.21458555e-03, 2.20831903e-02, ...,
        3.68133045e-04, 2.30351099e-02, 1.00000000e+00]])

# User + Item features

In [130]:
model = LightFM(loss = 'warp', no_components = 8)
model.fit(train, user_features = userfeature, item_features = count_matrix, epochs=25)

In [131]:
print(auc_score(model, test, train_interactions=train, user_features = userfeature,
            item_features = count_matrix).mean())
print(np.median(auc_score(model, test, train_interactions=train, user_features = userfeature,
                         item_features = count_matrix)))

print(recall_at_k(model, test, train_interactions=train,  user_features = userfeature,
            item_features = count_matrix).mean())
print(precision_at_k(model, test, train_interactions=train, user_features = userfeature,
            item_features = count_matrix).mean())

0.86253625
0.9260204
0.1971664428593215
0.021649744


# Recommendation Display

In [132]:
model.predict(np.array([0,0]), np.array([1, 1]))

array([-129.59239, -129.59239], dtype=float32)

In [133]:
users = [10, 11, 12]
items = [76, 75]
array_u = []
array_i = []
for user in users: 
    array_u += 2 * [user]
        
array_i = np.array([items]).repeat(3, axis=0).reshape(1,len(array_u))[0,:]
    #array_i.append([x] * 2)

In [134]:
array_u, array_i

([10, 10, 11, 11, 12, 12], array([76, 75, 76, 75, 76, 75]))

In [166]:
def get_recommendations(model, data, users, items):
    
    nu = len(users)
    ni = len(items)
    
    array_u = []
    array_i = []
    
    for user in users: 
        array_u += ni * [user]
        
    array_i = np.array([items]).repeat(nu, axis=0).reshape(1, len(array_u))[0,:]
  
    rec_scores = model.predict(np.array(array_u), array_i)
    
    re_reshape = rec_scores.reshape(nu, ni)
    
    for i in range(len(users)):
        
        print(users[i], ,'has a top choice:',
              data_user_item.columns[items[np.argmax(re_reshape[i])]])


In [167]:
get_recommendations(model, test, [10, 11], [76, 75])

10 has a top choice: P380805
11 has a top choice: P380805
